In [54]:
import numpy as np

# update import
%load_ext autoreload
%autoreload 2

In [2]:
# import msmd
from casatools import msmetadata 

measures_update ... acquiring the lock ... 


lock file is not empty : /Users/peijinz/.casa/data/data_update.lock
The lock file at /Users/peijinz/.casa/data is not empty.
A previous attempt to update path may have failed or exited prematurely.
Remove the lock file and set force to True with the desired version (default to most recent).
It may be best to completely repopulate path using pull_data and measures_update.
2024-12-19 21:33:20	SEVERE	::casa	lock file is not empty : /Users/peijinz/.casa/data/data_update.lock
2024-12-19 21:33:20	SEVERE	::casa	The lock file at /Users/peijinz/.casa/data is not empty.
2024-12-19 21:33:20	SEVERE	::casa	A previous attempt to update path may have failed or exited prematurely.
2024-12-19 21:33:20	SEVERE	::casa	Remove the lock file and set force to True with the desired version (default to most recent).
2024-12-19 21:33:20	SEVERE	::casa	It may be best to completely repopulate path using pull_data and measures_update.
2024-12-19 21:33:20	SEVERE	::casa	
2024-12-19 21:33:20	SEVERE	::casa	lock file is 

In [3]:
import numpy as np
from casatools import msmetadata
from astropy.time import Time

def print_ms_summary(vis):
    """
    Print a detailed summary of a measurement set using msmetadata tool
    
    Parameters:
    vis (str): Path to the measurement set
    """
    # Initialize the msmetadata tool
    msmd = msmetadata()
    msmd.open(vis)
    
    print(f"\n=== Summary for {vis} ===")
    
    # Basic observation info
    print("\nObservation Details:")
    print(f"Number of fields: {msmd.nfields()}")
    print(f"Number of scans: {msmd.nscans()}")
    print(f"Number of spectral windows: {msmd.nspw()}")
    print(f"Number of antennas: {msmd.nantennas()}")
    
    # Time information
    print("\nTime Range:")
    time_range = msmd.timerangeforobs(0)
    start_time = time_range['begin']['m0']['value']  # in days
    end_time = time_range['end']['m0']['value']      # in days
    duration_hours = (end_time - start_time) * 24.0  # convert days to hours
    
    # Get unique times using msmd.timesforfield
    times = msmd.timesforfield(0)  # assuming field 0
    ntimes = len(times)
    print(f"Number of unique times: {ntimes}")

    # Convert MJD to human readable format
    t_start = Time(start_time, format='mjd')
    t_end = Time(end_time, format='mjd')
    print(f"Start time (MJD): {start_time:.6f} ({time_range['begin']['refer']})")
    print(f"End time (MJD): {end_time:.6f} ({time_range['end']['refer']})")
    print(f"Start time (UTC): {t_start.iso}")
    print(f"End time (UTC): {t_end.iso}")
    print(f"Total time span: {duration_hours:.2f} hours")
    
    # Field information
    print("\nFields:")
    for field_id in range(msmd.nfields()):
        print(f"Field {field_id}: {msmd.fieldnames()[field_id]}")
        print(f"  Direction: {msmd.phasecenter(field_id)['m0']['value']:.3f}, "
              f"{msmd.phasecenter(field_id)['m1']['value']:.3f} (rad)")
    
    # Spectral Window information
    print("\nSpectral Windows:")
    for spw in range(msmd.nspw()):
        freq_range = msmd.chanfreqs(spw)
        print(f"SPW {spw}:  N_ch: {msmd.nchan(spw)} \t Freq: {freq_range[0]/1e9:.3f} - {freq_range[-1]/1e9:.3f} GHz \t Ch_w: {abs(msmd.chanwidths(spw)[0])/1e3:.1f} kHz \t Totbw: {msmd.bandwidths(spw)/1e6:.2f} MHz")

    # Antenna information
    print("\nAntennas:")
    print(f"Names: {', '.join(msmd.antennanames())}")
    
    # Close the tool
    msmd.done()

In [4]:

fname = './UDB20241215.ms'
print_ms_summary(fname)

2024-12-19 21:33:22	INFO	msmetadata_cmpt.cc::open	Performing internal consistency checks on ./UDB20241215.ms...
2024-12-19 21:33:22	INFO	MSMetaData::_computeScanAndSubScanProperties 	Computing scan and subscan properties...



=== Summary for ./UDB20241215.ms ===

Observation Details:
Number of fields: 1
Number of scans: 5
Number of spectral windows: 50
Number of antennas: 16

Time Range:
Number of unique times: 387
Start time (MJD): 60659.671464 (UTC)
End time (MJD): 60659.979809 (UTC)
Start time (UTC): 2024-12-15 16:06:54.500
End time (UTC): 2024-12-15 23:30:55.500
Total time span: 7.40 hours

Fields:
Field 0: Sun
  Direction: -1.688, -0.407 (rad)

Spectral Windows:
SPW 0:  N_ch: 30 	 Freq: 1.100 - 1.412 GHz 	 Ch_w: 10742.2 kHz 	 Totbw: 322.27 MHz
SPW 1:  N_ch: 22 	 Freq: 1.425 - 1.735 GHz 	 Ch_w: 14746.1 kHz 	 Totbw: 324.41 MHz
SPW 2:  N_ch: 13 	 Freq: 2.400 - 2.700 GHz 	 Ch_w: 25000.0 kHz 	 Totbw: 325.00 MHz
SPW 3:  N_ch: 12 	 Freq: 2.725 - 3.023 GHz 	 Ch_w: 27050.8 kHz 	 Totbw: 324.61 MHz
SPW 4:  N_ch: 11 	 Freq: 3.050 - 3.345 GHz 	 Ch_w: 29492.2 kHz 	 Totbw: 324.41 MHz
SPW 5:  N_ch: 10 	 Freq: 3.375 - 3.667 GHz 	 Ch_w: 32421.9 kHz 	 Totbw: 324.22 MHz
SPW 6:  N_ch: 9 	 Freq: 3.700 - 3.988 GHz 	 Ch_w: 3

In [61]:
# get time range for each scan
msmd = msmetadata()
msmd.open(fname)
t_range_msmd = msmd.timerangeforobs(0)

# get num of scans
n_times = msmd.timesforfield(0)
#msmd.close()

print(t_range_msmd)

{'begin': {'m0': {'unit': 'd', 'value': 60659.671464120234}, 'refer': 'UTC', 'type': 'epoch'}, 'end': {'m0': {'unit': 'd', 'value': 60659.97980902742}, 'refer': 'UTC', 'type': 'epoch'}}


2024-12-20 23:17:29	INFO	msmetadata_cmpt.cc::open	Performing internal consistency checks on ./UDB20241215.ms...


In [69]:
msmd.nscans

5

In [16]:
msmd.close()

True

In [17]:
import rotation_corr_util

In [15]:
rotation_corr_util.get_begin_end_time_from_ms(fname)

2024-12-20 03:38:22	INFO	msmetadata_cmpt.cc::open	Performing internal consistency checks on ./UDB20241215.ms...


(datetime.datetime(2024, 12, 15, 16, 6, 54, 499988),
 datetime.datetime(2024, 12, 15, 23, 30, 55, 499969))

In [ ]:
from eovsa_synop import rotation_corr_util
from eovsa_synop import wrap_wsclean